In [56]:
import pandas as pd
from espn_api.basketball import League
from datetime import datetime, timedelta
import pytz


In [50]:
league_id = 1800187247  
year = 2024

espn_s2 = 'AEC5UWfHuxcu831I73sNZEH4je3W%2FNNxVN4clK3USlazw8tVGSw2qLuTZdyLIKU6HlMpgsz0mZ0OMY%2F2SUwmALZ0YwRQBj41kvt3xawnde4YVncFCxfFz48ZUAORsacFlEvwQlPlbzTp3i%2BmUMALHNhQn5Yn%2Fhah9T5VSERo40psL0U30cbbZfxSoNirW4pueHAcykhIYumCo03uVMlEhObQIaDBBy9Kx2MLOBsGFuHymKx7ndXDPHhB4E1uWur9IGAWmF8GW8zq7NaEgr6DG4Bw'  # Replace with your ESPN_S2 cookie
swid = '{6E7237F4-5305-42FE-B237-F45305D2FE66}'        # Replace with your SWID cookie
league = League(league_id=league_id, year=year, espn_s2=espn_s2, swid=swid)


In [51]:
data = {
    "Team": ["Thunderstruck", "White Lightning", "Chef Curry", "Mark Will made it", "TJ's Tall Boys", "Bucksketball", "LameloBawl", "Bum Squad", "Tel Aviv Basketball Club", "Team Srivastava"],
    "Record": ["8-3", "8-3", "8-3", "6-5", "6-5", "5-6", "5-6", "4-7", "3-8", "2-9"]
}

df_new = pd.DataFrame(data)
df_new

,Team,Record
0,Thunderstruck,8-3
1,White Lightning,8-3
2,Chef Curry,8-3
3,Mark Will made it,6-5
4,TJ's Tall Boys,6-5
5,Bucksketball,5-6
6,LameloBawl,5-6
7,Bum Squad,4-7
8,Tel Aviv Basketball Club,3-8
9,Team Srivastava,2-9


In [ ]:
season_start_date = datetime(2023, 10, 24, tzinfo=pytz.timezone('US/Eastern'))  # Replace with your season start date

current_date = datetime.now(pytz.timezone('US/Eastern'))

num_weeks = ((current_date - season_start_date).days // 7) + 1

In [52]:
cumulative_points = {team.team_name: 0 for team in league.teams}

for week in range(1, num_weeks + 1):
    boxscores = league.box_scores(week)

    for boxscore in boxscores:
        cumulative_points[boxscore.home_team.team_name] += boxscore.home_score
        cumulative_points[boxscore.away_team.team_name] += boxscore.away_score


In [53]:
cumulative_points_against = {team.team_name: 0 for team in league.teams}


for week in range(1, num_weeks + 1):
    boxscores = league.box_scores(week)

    for boxscore in boxscores:
        cumulative_points_against[boxscore.home_team.team_name] += boxscore.away_score
        cumulative_points_against[boxscore.away_team.team_name] += boxscore.home_score

In [54]:
df_new['PF'] = df_new['Team'].map(cumulative_points)
df_new['PA'] = df_new['Team'].map(cumulative_points_against)

df_new


,Team,Record,PF,PA
0,Thunderstruck,8-3,16526.0,15143.0
1,White Lightning,8-3,15886.0,14757.0
2,Chef Curry,8-3,15084.0,14653.0
3,Mark Will made it,6-5,15097.0,13787.0
4,TJ's Tall Boys,6-5,14738.0,14839.0
5,Bucksketball,5-6,15569.0,14773.0
6,LameloBawl,5-6,13377.0,14191.0
7,Bum Squad,4-7,13838.0,15582.0
8,Tel Aviv Basketball Club,3-8,14214.0,15160.0
9,Team Srivastava,2-9,14031.0,15475.0


In [55]:
weekly_points_scored = {team.team_name: [] for team in league.teams}
weekly_points_against = {team.team_name: [] for team in league.teams}

df_new['Total Weeks'] = df_new['Record'].apply(lambda x: sum(map(int, x.split('-'))))

max_weeks = df_new['Total Weeks'].max()

df_new['PF/Wk'] = df_new['Team'].map(cumulative_points).div(df_new['Total Weeks'])
df_new['PA/Wk'] = df_new['Team'].map(cumulative_points_against).div(df_new['Total Weeks'])

df_rounded = df_new.round()
df_rounded


,Team,Record,PF,PA,Total Weeks,PF/Wk,PA/Wk
0,Thunderstruck,8-3,16526.0,15143.0,11,1502.0,1377.0
1,White Lightning,8-3,15886.0,14757.0,11,1444.0,1342.0
2,Chef Curry,8-3,15084.0,14653.0,11,1371.0,1332.0
3,Mark Will made it,6-5,15097.0,13787.0,11,1372.0,1253.0
4,TJ's Tall Boys,6-5,14738.0,14839.0,11,1340.0,1349.0
5,Bucksketball,5-6,15569.0,14773.0,11,1415.0,1343.0
6,LameloBawl,5-6,13377.0,14191.0,11,1216.0,1290.0
7,Bum Squad,4-7,13838.0,15582.0,11,1258.0,1417.0
8,Tel Aviv Basketball Club,3-8,14214.0,15160.0,11,1292.0,1378.0
9,Team Srivastava,2-9,14031.0,15475.0,11,1276.0,1407.0
